In [ ]:
import numpy as np
import os
import glob
import cv2
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mpl
import bat_functions as bf

In [ ]:
plot_folder = '.../bats-data/plots'
save_folder = os.path.join(plot_folder, 'wingbeats')

In [ ]:
# frame_root_folder = '/media/golden/Elements/bats'
# date = '16Nov'
# camera = 'FibwePublic'
# frame_files = sorted(glob.glob(os.path.join(frame_root_folder, date, camera, '*/*.jpg')))
# len(frame_files)

In [ ]:
tracks = raw_tracks[date][camera]

In [ ]:
track_lengths = [len(track['track']) for track in tracks]

In [ ]:
track_lengths = sorted(track_lengths)
hist = np.histogram(track_lengths)
num_lengths = hist[0]
bins = hist[1]
plt.scatter(bins[1:], num_lengths)
cum_lengths = np.cumsum(num_lengths)
plt.figure()
plt.plot(bins[1:], cum_lengths / cum_lengths[-1])

In [ ]:
frame_ind = 15000
frame = plt.imread(frame_files[frame_ind])

In [ ]:
# raw_tracks[date][observation_name][0]['contour']

In [ ]:
def get_peaks(track):
    if not 'rects' in track.keys():
        track['rects'] = get_rects(track)
                    
    max_edge = np.nanmax(track['rects'], 1)
    max_edge = max_edge[~np.isnan(max_edge)]
    peaks = signal.find_peaks(max_edge)[0]
    
    return peaks, max_edge

In [ ]:
frame_root_folder = '.../Elements/bats'
date = '17Nov'
glob.glob(os.path.join(frame_root_folder, date, '*'))

In [ ]:
cameras_frame_shift = {'16Nov':{'Chyniangale': 0,
               'BBC': -1209,
               'FibwePublic': 0,
               'MusoleTower': 0,
               'Puku': 0,
               'FibweParking': 0,
               'MusoleParking': 0,
               'MusolePath': 0,
               'Sunset': 0,
               'NotChyniangale': 0,
               },
               '17Nov':{'Chyniangale': 0,
               'BBC': 0,
               'FibwePublic': 0,
               'MusoleTower': 0,
               'Puku': 0,
               'FibweParking2': -805,
               'MusoleParking': 0,
               'MusolePath2': 0,
               'Sunset': 0,
               'NotChyniangale': 0,
               }
              }

In [ ]:
save_folder = '.../bats-data/wing-validation'
os.makedirs(os.path.join(save_folder, 'validation-images'), exist_ok=True)
os.makedirs(os.path.join(save_folder, 'validation-csvs'), exist_ok=True)
             

    
num_tracks = 0
shift = 0
track_width = 3
draw_tracks = False
draw_contours = False

save_annotation_ims = True
show_plots = False

border = 15

samples_per_obs = 100

np.random.seed(1993)

frame_root_folder = '.../Elements/bats'
root_folder = '.../kasanka-bats/processed/deep-learning'
raw_tracks = {}
for date in ['16Nov', '17Nov']:
    day_folder = os.path.join(root_folder, date)
    raw_tracks[date] = {}
    track_files = sorted(
        glob.glob(os.path.join(day_folder, '*/crossing_tracks.npy'))
    )
    for track_file in track_files:
        camera = track_file.split('/')[-2]
        raw_track_list = np.load(track_file, allow_pickle=True)
        raw_tracks[date][camera] = raw_track_list

    camera_folders = sorted(
        glob.glob(os.path.join(frame_root_folder, date, '*'))
    )
    cameras = [os.path.basename(cf) for cf in camera_folders]
    for camera in cameras:
        if 'test' in camera:
            continue
        frame_files = sorted(
            glob.glob(os.path.join(frame_root_folder, date, camera, '*/*.jpg'))
        )

        track_list = raw_tracks[date][camera]
        inds = np.random.choice(len(track_list), 
                                samples_per_obs , 
                                replace=False)

        
        
        frame_shift = cameras_frame_shift[date][camera]
        for track_ind, track in zip(inds, track_list[inds]):

            try:
                wing_peaks, max_edge = get_peaks(track)
            #     print(wing_peaks)
            #     plt.figure()
            #     plt.plot(max_edge)
            #     plt.scatter(wing_peaks, max_edge[wing_peaks])

                if wing_peaks.size:
                    rel_frame = np.random.choice(wing_peaks)
                else:
                    rel_frame = np.random.randint(0, track['track'].shape[0])
                    print('rel frame', rel_frame)
                frame_ind = track['first_frame'] + rel_frame - frame_shift
                frame = plt.imread(frame_files[frame_ind])
                contour = track['contour'][rel_frame]
                rect = cv2.minAreaRect(contour)
                box = cv2.boxPoints(rect)

                xmin = np.min(contour[:,0]) - border
                ymin = np.min(contour[:,1]) - border
                xmax = np.max(contour[:,0]) + border
                ymax = np.max(contour[:,1]) + border

                crop = frame[ymin-border:ymax+border, xmin-border:xmax+border]
                bright_crop = np.copy(crop)
                bias = np.min(bright_crop)
                bright_crop -= bias
                scale = np.max(bright_crop)
                bright_crop = bright_crop / scale
                crop_shape = bright_crop.shape

                if save_annotation_ims:
                    file_name = f'date_{date}_camera_{camera}_frame_{frame_ind}_track_{track_ind}_x_{xmin}_y_{ymin}'
                    save_file = os.path.join(save_folder, 'validation-images', file_name + '.png')
                    bright_crop_bgr = cv2.cvtColor(bright_crop.astype(np.float32), 
                                                     cv2.COLOR_BGR2RGB)
                    cv2.imwrite(save_file, bright_crop_bgr*255)
                    crop_info = pd.DataFrame({'date': date,
                                              'camera': camera,
                                              'frame': frame_ind,
                                              'xmin': xmin,
                                              'ymin': ymin,
                                              'xmax': xmax,
                                              'ymax': ymax,
                                              'border': border,
                                              'hard': False,
                                              'wing_estimate': np.max(rect[1]),
                                              'wingtip1_x': -1,
                                              'wingtip1_y': -1,
                                              'wingtip2_x': -1,
                                              'wingtip2_y': -1,
                                             },
                                            index=[0]
                                            )
                    crop_info_file = os.path.join(save_folder, 'validation-csvs', file_name + '.csv')
                    crop_info.to_csv(crop_info_file, index=False)


                if show_plots:
                    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
                    axs[0].imshow(crop)
                    axs[1].imshow(bright_crop)
                    if draw_contours:
                        if len(contour.shape) > 1:
                            rect = cv2.minAreaRect(contour)
                            box = cv2.boxPoints(rect)
                            box_d = np.int0(box)
                            cv2.drawContours(frame, [box_d], 0, (0,255,100), 1)
                            cv2.drawContours(frame, [contour], 0, (255,255,100), 1)
                    rescaled = (frame[ymin-border:ymax+border, xmin-border:xmax+border]-bias) / scale
                    rescaled = np.minimum(rescaled, 1.0)
                    axs[2].imshow(rescaled)
            except:
                print(f'issue with {date} {camera} track {track_ind}')
# fig.savefig()

In [ ]:
inds

In [ ]:
pd.DataFrame(d, index=[0])

In [ ]:
x = (frame[ymin-border:ymax+border, xmin-border:xmax+border]-bias) / scale
print(x.min(), x.max())

In [ ]:
frame.shape[0]-ymax-20, frame.shape[0]-(ymin) +20

In [ ]:
wing_peaks